In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import utils as ut
import m_models as mm
import os
from tqdm import tqdm_notebook as tqdm

import pylab

import statsmodels.api as sm

import scipy.stats as sps
from scipy.optimize import minimize_scalar as minimize_scalar

import pickle

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markersize'] = 12
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 40
plt.rcParams['axes.titlesize']=36
plt.rcParams['axes.labelsize']=40
plt.rcParams['figure.figsize'] = (24.0, 12.0)
plt.rcParams['font.size'] = 40

In [ ]:
X, y = ut.DataLoader(name = 'boston')
X, y = ut.preprocess(X, y)

In [ ]:
real_data = mm.ALC(X, y, progress_bar=lambda x: tqdm(x, leave = False))

In [9]:
statmodel = ut.define_model(y)

In [10]:
sample_size = np.arange(10, X.shape[0], 5)
sample_size = np.arange(10, 500, 5)

In [11]:
List_of_E = []

for m in tqdm(sample_size):
    listt = []
    for _ in range(100):
        X_m, y_m = ut.get_subset(X, y, m)
        model = statmodel(y_m, X_m)
        w_hat = ut.get_params(X_m, y_m)
        listt.append(model.covariance(w_hat))
    List_of_E.append(np.diagonal(np.mean(listt, axis = 0)))
    
List_of_E = np.asarray(List_of_E)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
M = 200
X_M, y_M = ut.get_subset(X, y, M, duplications=False)
w_hat = ut.get_params(X_M, y_M)

In [13]:
List_of_answ = []

for i in tqdm(range(1000)):
    X_m, y_m = ut.get_subset(X_M, y_M, M//2, duplications=False)
    model = statmodel(y_m, X_m)
    w_hat = ut.get_params(X_m, y_m)
    List_of_answ.append(-model.hessian_fixed(w_hat)/X_m.shape[0])

List_of_answ = np.asarray(List_of_answ)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
approx = np.diagonal(np.linalg.inv(np.mean(List_of_answ, axis = 0)))

i_matrix = np.linalg.inv(np.mean(List_of_answ, axis = 0))

In [15]:
approx

array([1.03052085, 1.30681911, 2.30828735, 1.41989758, 3.46021023,
       1.76144229, 3.23880287, 7.01391536, 7.70932699, 1.98662069,
       1.85812569, 2.73004251])

In [ ]:
r = 30
k = 4
res = approx[k]/sample_size

plt.plot(sample_size[r:], List_of_E[r:, k])

plt.plot(sample_size[r:], res[r:])

In [ ]:
def aDistribution(cov, alpha=0.05):
    
    W = sps.multivariate_normal(mean=np.zeros(cov.shape[0]), cov = cov).rvs(size=1000)

    function = lambda r: (((W**2).sum(axis=1) > r**2).mean() - alpha)**2

    res = minimize_scalar(function, method='Golden')['x']
    return res

In [ ]:
list_of_r = []

sample_sizes = np.arange(10, 5000, 5)

for m in tqdm(sample_sizes):
    list_of_r.append(aDistribution(i_matrix/m))
    
list_of_r = np.asarray(list_of_r)

In [ ]:
r = 10

plt.plot(real_data['m'][r:], real_data['E'][r:])

plt.plot(sample_sizes[r:], list_of_r[r:])

In [ ]:
sample_sizes[np.where(list_of_r < 0.25)][0]